In [40]:
import numpy as np
import pandas as pd

In [41]:
with open("../twitter-datasets/twitter-datasets/train_neg.txt") as f:
    neg_txt = [line for line in f]
    neg_df = pd.DataFrame(neg_txt, columns=['line']).drop_duplicates(keep='first')
    print(neg_df.head())
f.close()

                                                line
0  vinco tresorpack 6 ( difficulty 10 of 10 objec...
1  glad i dot have taks tomorrow ! ! #thankful #s...
2  1-3 vs celtics in the regular season = were fu...
3  <user> i could actually kill that girl i'm so ...
4  <user> <user> <user> i find that very hard to ...


In [42]:
with open("../twitter-datasets/twitter-datasets/train_pos.txt") as f:
    pos_txt = [line for line in f]
    pos_df = pd.DataFrame(pos_txt, columns=['line']).drop_duplicates(keep='first')
    print(pos_df.head())
f.close

                                                line
0  <user> i dunno justin read my mention or not ....
1  because your logic is so dumb , i won't even c...
2  " <user> just put casper in a box ! " looved t...
3  <user> <user> thanks sir > > don't trip lil ma...
4  visiting my brother tmr is the bestest birthda...


<function TextIOWrapper.close()>

In [43]:
pos_df['label'] = 1
neg_df['label'] = 0

In [44]:
full_df = pd.concat([pos_df,neg_df],ignore_index=True)
full_df.tail()

,line,label
181316,can't wait to fake tan tonight ! hate being pa...,0
181317,<user> darling i lost my internet connection ....,0
181318,kanguru defender basic 4 gb usb 2.0 flash driv...,0
181319,rizan is sad now\n,0
181320,"no text back ? yea , he mad\n",0


In [45]:
import re
!pip install nltk
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [47]:
from tqdm import tqdm
corpus = []
for i in tqdm(range(len(full_df))):
  review = re.sub('[^a-zA-Z]', ' ', full_df['line'][i]) 
  review = review.lower() 
  review = review.split() 
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)


100%|██████████| 181321/181321 [01:23<00:00, 2162.42it/s]


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features= 1500)
X = tv.fit_transform(corpus).toarray()
y = full_df.iloc[:,-1].values

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = full_df.iloc[:,-1].values

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from  sklearn.neighbors import KNeighborsClassifier
classifiers = [GaussianNB(),LogisticRegressionCV(),RandomForestClassifier(),KNeighborsClassifier()]
for classifier in classifiers:
    classifier.fit(X_train, y_train)
    print(accuracy_score(classifier.predict(X_test),y_test))

0.666375


C:\Users\lucas\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lucas\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mo

0.769325
0.77995
0.7211


In [52]:
classifier = classifiers[3]
accuracy_score(classifier.predict(X_test),y_test)

0.5210533572314904

In [9]:
#!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model_embd = SentenceTransformer("bert-base-nli-mean-tokens").to('cuda:0')

AssertionError: Torch not compiled with CUDA enabled

In [8]:
from tqdm import tqdm
embedings = []
for i in tqdm(range(len(full_df))):
    embedings.append(model_embd.encode(full_df['line'][i]))

  1%|          | 1210/200000 [00:45<2:04:31, 26.61it/s]


KeyboardInterrupt: 

In [23]:
import numpy as np